In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
trips = pd.read_csv('../data/raw/201501-hubway-tripdata.csv')
trips = trips.append(pd.read_csv('../data/raw/201502-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201503-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201504-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201505-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201506-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201507-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201508-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201509-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201510-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201511-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201512-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201601-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201602-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201603-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201604-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201605-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201606-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201607-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201608-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201609-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201610-hubway-tripdata.csv'))
trips = trips.append(pd.read_csv('../data/raw/201611-hubway-tripdata.csv'))
print(trips.shape)

(2313770, 15)


In [3]:
trips.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [4]:
#drop unwanted features and rename station features
trips = trips.drop(['tripduration', 'bikeid', 'usertype', 'birth year', 'gender'], axis=1)
trips = trips.rename(columns={'start station id': 'start_station_id', 
                              'start station name': 'start_station_name',
                              'start station latitude': 'start_station_latitude',
                              'start station longitude': 'start_station_longitude',
                              'end station id': 'end_station_id', 
                              'end station name': 'end_station_name',
                              'end station latitude': 'end_station_latitude',
                              'end station longitude': 'end_station_longitude'})

In [5]:
trips.dtypes

starttime                   object
stoptime                    object
start_station_id             int64
start_station_name          object
start_station_latitude     float64
start_station_longitude    float64
end_station_id              object
end_station_name            object
end_station_latitude        object
end_station_longitude       object
dtype: object

In [6]:
#coerce all end station ids to numbers. if not possible, then set NaN
trips['end_station_id'] = pd.to_numeric(trips['end_station_id'], errors='coerce')
print('Number of trips with unknown end station ID:', trips[pd.isnull(trips['end_station_id'])].shape[0])
trips = trips[pd.notnull(trips['end_station_id'])]
trips = trips.astype({'start_station_id': int, 'end_station_id': int})

Number of trips with unknown end station ID: 1


In [7]:
#station ID 1 is Hubway's warehouse and can be removed
warehouse_trips = (trips['start_station_id'] == 1) | (trips['end_station_id'] == 1)
print('Number of trips with station ID = 1:', trips[warehouse_trips].shape[0])
trips = trips[warehouse_trips == False]

Number of trips with station ID = 1: 486


In [8]:
trips['end_station_latitude'] = pd.to_numeric(trips['end_station_latitude'], errors='coerce')
print('Number of trips with unknown end station latitude:', trips[pd.isnull(trips['end_station_latitude'])].shape[0])
trips['end_station_longitude'] = pd.to_numeric(trips['end_station_longitude'], errors='coerce')
print('Number of trips with unknown end station longitude:', trips[pd.isnull(trips['end_station_longitude'])].shape[0])

Number of trips with unknown end station latitude: 0
Number of trips with unknown end station longitude: 0


In [9]:
trips['starttime'] = trips['starttime'].apply(pd.Timestamp)
trips['stoptime'] = trips['stoptime'].apply(pd.Timestamp)
trips.dtypes

starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start_station_id                    int32
start_station_name                 object
start_station_latitude            float64
start_station_longitude           float64
end_station_id                      int32
end_station_name                   object
end_station_latitude              float64
end_station_longitude             float64
dtype: object

In [10]:
#remove trips from or to "virtual" locations outside Boston
BOSTON_LATITUDE = 42.355428
BOSTON_LONGITUDE = -71.069786
out_of_bounds = ((trips['start_station_latitude'] > (BOSTON_LATITUDE + 1)) | 
                 (trips['start_station_latitude'] < (BOSTON_LATITUDE - 1)) | 
                 (trips['end_station_latitude'] > (BOSTON_LATITUDE + 1)) | 
                 (trips['end_station_latitude'] < (BOSTON_LATITUDE - 1)) |
                 (trips['start_station_longitude'] > (BOSTON_LONGITUDE + 1)) | 
                 (trips['start_station_longitude'] < (BOSTON_LONGITUDE - 1)) | 
                 (trips['end_station_longitude'] > (BOSTON_LONGITUDE + 1)) | 
                 (trips['end_station_longitude'] < (BOSTON_LONGITUDE - 1)))
print("Number of trips with station locations outside of Boston: ", trips[out_of_bounds].shape[0])
trips = trips[out_of_bounds == False]

Number of trips with station locations outside of Boston:  3


In [11]:
#remove trips that end in December 2016
print("Number of trips ending in December 2016: ", trips[trips['stoptime'] >= pd.Timestamp(2016, 12, 1, 0, 0, 0)].shape[0])
trips = trips[trips['stoptime'] < pd.Timestamp(2016, 12, 1, 0, 0, 0)]

Number of trips ending in December 2016:  9


In [12]:
trips.head()

,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude
0,2015-01-01 00:21:44,2015-01-01 00:30:47,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075
1,2015-01-01 00:27:03,2015-01-01 00:34:21,80,MIT Stata Center at Vassar St / Main St,42.361962,-71.092053,95,Cambridge St - at Columbia St / Webster Ave,42.372969,-71.094445
2,2015-01-01 00:31:31,2015-01-01 00:35:46,91,One Kendall Square at Hampshire St / Portland St,42.366277,-71.091690,68,Central Square at Mass Ave / Essex St,42.365070,-71.103100
3,2015-01-01 00:53:46,2015-01-01 01:00:58,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075
4,2015-01-01 01:07:06,2015-01-01 01:19:21,105,Lower Cambridgeport at Magazine St/Riverside Rd,42.356954,-71.113687,88,Inman Square at Vellucci Plaza / Hampshire St,42.374035,-71.101427


In [13]:
print("Number of trips:", trips.shape[0])

Number of trips: 2313271


In [14]:
trips = trips.set_index('starttime')
trips.to_csv('../data/raw/201501-201611-hubway-tripdata.csv')